## Loading the Readings and the Processed Readings to the Mongo Database

Many libraries are required for this step. Most of these are required for converting the sensor readings into magnetometer values. The aberystwyth magnetometer samba server is also required.

In [1]:
import os

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn import mixture
from datetime import datetime, timedelta

from IPython.display import display

import frg_single_fgm3_dat_insert as frg_insert

from cpdetect import cpDetector

from pymongo import MongoClient
import mongo_cursors

import json
import igrf12

# Set the path of the mag_data samba share mount, or path to a copy of the samba share data.
dat_fp = "../mag_data/"

# Setup the time for converting NI time to UNIX time.
NI_zerotime = np.datetime64('1904-01-01T00:00:00.000Z').astype('datetime64[s]').astype(np.float64)
NI_zerotime

/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


-2082844800.0

Load all the pre-2014 data, with corrected times.

In [2]:
def load_2013_dat():
    """Loads data from the 2013 folder, in the Aberystwyth magnetometer samba share."""
    hour_change_2011 = {"start": datetime(year=2011, month=3, day=27, hour=1, minute=0, second=0),
                        "end": datetime(year=2011, month=10, day=30, hour=2, minute=0, second=0)}
    hour_change_2012 = {"start": datetime(year=2012, month=3, day=25, hour=1, minute=0, second=0),
                        "end": datetime(year=2012, month=10, day=28, hour=2, minute=0, second=0)}
    hour_change_2013 = {"start": datetime(year=2013, month=3, day=31, hour=1, minute=0, second=0),
                        "end": datetime(year=2013, month=10, day=27, hour=2, minute=0, second=0)}

    dataframes = []
    
    for root, dirs, files in os.walk(dat_fp+"2013"):
        print("Files loaded:")
        for file in files:
            if file.lower().endswith(".csv"):
                try:
                    file = os.path.join(root, file)
                    # Retrieve file modification time.
                    date = datetime.fromtimestamp(os.path.getmtime(file))
                    day_dat = pd.read_csv(file, names=("time", "reading", "temperature"))
                    print(file)
                except Exception as e:
                    print(e)
                else:
                    # First, set the time to an integer, then to a string.
                    day_dat["time"] = day_dat["time"].astype('int').astype("str", copy=False)
                    # Make sure all strings are 6 characters long, filled with zeros
                    day_dat["time"] = day_dat["time"].apply(lambda dt : dt.zfill(6))
                    # Convert strings to datetimes, formatted as "%H%M%S".
                    day_dat["time"] = pd.to_datetime(day_dat["time"], format="%H%M%S")
                    # Include the current year, day, month in the datetimes, as retrieved from the file modification time.
                    day_dat["time"] = day_dat["time"].apply(lambda dt: dt.replace(year=date.year, month=date.month, day=date.day))
                    # Due to an error, the last recording in the csv's is for the next day.
                    # The error can be verified by checking whether the last time is
                    # earlier than the previous.
                    # We need a try-except because the csv may only contain 1 item.
                    try:
                        if day_dat.iloc[-1, 0] < day_dat.iloc[-2, 0]:
                            # if the last time is earlier than the previous, then add a day to the time.
                            day_dat.iloc[-1, 0] += timedelta(days=1)
                    except IndexError:
                        print("Warning! Only 1 reading in the csv!")
                        
                    dataframes.append(day_dat)

    sensor_dat = pd.concat(dataframes, ignore_index=True)
    # Readings were taken in GMT and BST. Make them all GMT (UTC).
    # Find BST times.
    BST_mask = ((sensor_dat["time"] >= hour_change_2011["start"]) & (sensor_dat["time"] < hour_change_2011["end"]))
    BST_mask |= ((sensor_dat["time"] >= hour_change_2012["start"]) & (sensor_dat["time"] < hour_change_2012["end"]))
    BST_mask |= ((sensor_dat["time"] >= hour_change_2013["start"]) & (sensor_dat["time"] < hour_change_2013["end"]))
    print(sensor_dat.shape, BST_mask.shape)
    # Subtract an hour from the bst times
    sensor_dat.loc[BST_mask, "time"] -= timedelta(hours=1)
    sensor_dat.set_index("time", inplace=True)
    
    return sensor_dat

all_mag_dat = load_2013_dat()
all_mag_dat.head()

Files loaded:
../mag_data/2013/3000000.CSV
../mag_data/2013/2020000.CSV
../mag_data/2013/2000000.CSV
../mag_data/2013/1960000.CSV
../mag_data/2013/0810000.CSV
../mag_data/2013/3280000.CSV
../mag_data/2013/0321202.CSV
../mag_data/2013/0830000.CSV
../mag_data/2013/0820000.CSV
../mag_data/2013/0310000.CSV
../mag_data/2013/1360000.CSV
../mag_data/2013/2070000.CSV
../mag_data/2013/0550000.CSV
../mag_data/2013/0930000.CSV
../mag_data/2013/2300000.CSV
../mag_data/2013/2290000.CSV
../mag_data/2013/3560000.CSV
../mag_data/2013/1260000.CSV
../mag_data/2013/2720000.CSV
../mag_data/2013/0900000.CSV
../mag_data/2013/2350000.CSV
../mag_data/2013/3170000.CSV
../mag_data/2013/2880000.CSV
../mag_data/2013/1920000.CSV
../mag_data/2013/1730000.CSV
../mag_data/2013/0691221.CSV
../mag_data/2013/1340000.CSV
../mag_data/2013/1970000.CSV
../mag_data/2013/3610000.CSV
../mag_data/2013/2420000.CSV
../mag_data/2013/1090000.CSV
../mag_data/2013/2900000.CSV
../mag_data/2013/2540000.CSV
../mag_data/2013/3220000.CSV


../mag_data/2013/0490000.CSV
../mag_data/2013/2110000.CSV
../mag_data/2013/1110000.CSV
../mag_data/2013/2010000.CSV
../mag_data/2013/1470000.CSV
../mag_data/2013/1010000.CSV
../mag_data/2013/3360000.CSV
../mag_data/2013/2220000.CSV
../mag_data/2013/2760000.CSV
../mag_data/2013/0280000.CSV
../mag_data/2013/1130000.CSV
../mag_data/2013/0990000.CSV
../mag_data/2013/1980000.CSV
../mag_data/2013/0350000.CSV
../mag_data/2013/1100000.CSV
../mag_data/2013/0351542.CSV
Warning! Only 1 reading in the csv!
../mag_data/2013/2710000.CSV
../mag_data/2013/2260000.CSV
../mag_data/2013/2970000.CSV
../mag_data/2013/2580000.CSV
../mag_data/2013/0190000.CSV
../mag_data/2013/0710000.CSV
../mag_data/2013/1760000.CSV
../mag_data/2013/2910000.CSV
../mag_data/2013/2930000.CSV
../mag_data/2013/0850000.CSV
../mag_data/2013/0910000.CSV
../mag_data/2013/2240000.CSV
../mag_data/2013/0570000.CSV
../mag_data/2013/1460000.CSV
../mag_data/2013/2560000.CSV
../mag_data/2013/2200000.CSV
../mag_data/2013/3350000.CSV
../mag_

,reading,temperature
time,,
2011-10-27 23:00:05,31896.0,21.343
2011-10-27 23:00:08,31896.0,21.342
2011-10-27 23:00:11,31896.0,21.333
2011-10-27 23:00:14,31896.0,21.346
2011-10-27 23:00:17,31896.0,21.356


Load all magnetometer data from after 2014.

In [3]:
def load_data(folder):
    """Loads post 2013 data from the Aberystwyth magnetometer server."""
    #year = datetime(int(folder),1,1)
    dataframes = []
    
    for root, dirs, files in os.walk(folder):
        for file in files:
            try:
                day_dat = pd.read_csv(os.path.join(root, file),
                                      names=("time", "reading", "temperature"),
                                    dtype={"time": np.float64, "reading": np.float64, "temperature": np.float64})
            except Exception as e:
                print("Could not load:", os.path.join(root, file))
                print(e)
            else:
                # Convert NI time to UNIX time.
                day_dat["time"] = pd.to_datetime(day_dat["time"] + NI_zerotime, unit='s')
                dataframes.append(day_dat)

    sensor_dat = pd.concat(dataframes, ignore_index=True)
    sensor_dat.sort_values("time", inplace=True)
    sensor_dat.set_index("time", inplace=True)

    return sensor_dat


for year_data in ["2014", "2015", "2016", "2017", "2018"]:
    all_mag_dat = all_mag_dat.append(load_data(dat_fp+year_data))
    
all_mag_dat = all_mag_dat.loc[pd.notnull(all_mag_dat.index)]
display(all_mag_dat.head())
display(all_mag_dat.tail())

Could not load: ../mag_data/2014/data.zip
('Multiple files found in compressed zip file %s', "['DATA059.csv', 'DATA060.csv', 'DATA061.csv', 'DATA062.csv', 'DATA063.csv', 'DATA064.csv', 'DATA065.csv', 'DATA066.csv', 'DATA067.csv', 'DATA068.csv', 'DATA069.csv', 'DATA070.csv', 'DATA071.csv', 'DATA072.csv', 'DATA073.csv', 'DATA074.csv', 'DATA075.csv', 'DATA076.csv', 'DATA077.csv', 'DATA078.csv', 'DATA079.csv', 'DATA080.csv', 'DATA081.csv', 'DATA082.csv', 'DATA083.csv', 'DATA085.csv', 'DATA086.csv', 'DATA087.csv']")
Could not load: ../mag_data/2014/.directory
could not convert string to float: 'ViewMode=1'


,reading,temperature
time,,
2011-10-27 23:00:05,31896.0,21.343
2011-10-27 23:00:08,31896.0,21.342
2011-10-27 23:00:11,31896.0,21.333
2011-10-27 23:00:14,31896.0,21.346
2011-10-27 23:00:17,31896.0,21.356


,reading,temperature
time,,
2018-06-30 09:28:56.994999886,47453.0,2.704
2018-06-30 09:29:00.049000263,47455.0,2.700
2018-06-30 09:29:03.094999790,47456.0,2.707
2018-06-30 09:29:06.150000095,47455.0,2.703
2018-06-30 09:29:09.200999737,47454.0,2.712


Load the preprocessed raw data to the database.

In [4]:
# Reset the index, so that timestamps are included in the upload.
frg_insert.write_raw_dat(all_mag_dat.reset_index().to_dict('records'))

### Process raw data into magnetometer data, and add IGRF data.

In [5]:
mag_dat_1h = all_mag_dat.resample('1h').mean()
mag_dat_1h.dropna(inplace=True)
all_mag_dat = all_mag_dat.resample('1T').mean()
all_mag_dat.dropna(inplace=True)

## Splitting the data into 
detector = cpDetector([mag_dat_1h["reading"]], distribution='log_normal', log_odds_threshold=0)
detector.detect_cp()
# Get the changepoint times.
cp_times = mag_dat_1h.index[detector.change_points["traj_0"].ts]
# set the minimum difference for a time discontinuity to be considered "significant". 
sig_diff = np.timedelta64(1, 'D')
# Select the times where there are significant discontinuities.
discs = all_mag_dat.dropna().index[np.append([False], np.diff(all_mag_dat.dropna().index) > sig_diff)]
# Combine the changepoint and discontinuity times using a union.
split_times = np.union1d(cp_times, discs)
# Sort the times.
split_times.sort()
# Append the very first time from all the magnetometer data and the very last time.
# This is so when performing calculations, the magnetometer data between the very
# start/ end and the first/ last changepoint/ discontinuity is included.
split_times = np.append([np.datetime64(all_mag_dat.index[0])], split_times)
split_times = np.append(split_times, [np.datetime64(all_mag_dat.index[-1])])

## Setting up the data for converting into magnetometer data.
# Set the bias.
bias = 90000
# Calculate the period.
all_mag_dat["period"] = np.power(((-all_mag_dat["reading"]*70/65534+120)*1000+bias), -1)
# Create a place to store the "corrected" temperature.
all_mag_dat["temp_corr"] = all_mag_dat.loc[:, "temperature"]
# create places to store the period with the temperature correction and the magnetic field.
all_mag_dat["period_temp_corr"] = np.nan
all_mag_dat["F"] = np.nan

# Set Timur Zagidulin's function which converts the "period" readings into magnetometer readings (in nT).
coefs = np.vstack(([ 7.75627250e+06, -5.60893614e+06, 1.61438337e+06, -2.30773456e+05,
                    1.64006851e+04, -4.67419417e+02],
                     [ 1.37845775e+07, -1.08586181e+07, 3.43274556e+06, -5.43192224e+05,
                      4.30466903e+04, -1.37019412e+03],
                     [ 9.93009467e+06, -7.05978073e+06, 1.98324201e+06, -2.74458051e+05,
                      1.87117495e+04, -5.06023333e+02])).mean(axis=0)

def mag_field(period_dat, coefs=coefs):
    return np.polynomial.polynomial.polyval(period_dat*1e6, coefs)

## Perform corrections and conversions.
def quad_fun(x, a, b, c):
    return a*x*x + b*x + c


def corr_temp(mag_dat):
    X = np.column_stack((mag_dat["temperature"], mag_dat["period"]))
    gmm = mixture.GaussianMixture(n_components=2,
                                  covariance_type='full',
                                  means_init=((15, 0.000012), (0, 0.00001)),
                                 random_state=100).fit(X)

    temp_loopbacks = gmm.predict(X).astype(np.bool)
    mag_dat.loc[temp_loopbacks, "temp_corr"] += mag_dat["temperature"].max()
    return mag_dat["temp_corr"], temp_loopbacks


def corr_period_temp(mag_dat, fun):
#     popt, pcov = curve_fit(fun, mag_dat["temp_corr"],
#                      mag_dat["period"],
#                      p0=(1e-7, -1e-2, 1e-6),
#                      bounds=((0,-1e6,-1e6), (1e6, 0, 1e6)),
#                      maxfev=1000)
    popt, pcov = curve_fit(fun, mag_dat["temp_corr"], mag_dat["period"], maxfev=1000)
    temp_period_fit = fun(mag_dat["temp_corr"], *popt)
    mag_dat["period_temp_corr"] = mag_dat["period"] - temp_period_fit + mag_dat["period"].mean()
    return popt, pcov, mag_dat["period_temp_corr"]


def plot_results(mag_dat, popt, temp_loopbacks, fun):
    temp_period_fit = fun(mag_dat["temp_corr"], *popt)
    fig, ax = plt.subplots(1,2,figsize=(15,5))
    ax[0].set_title("Temperature vs Period between\n" + str(mag_dat.index.min()) + " and " + str(mag_dat.index.max()))
    ax[0].set_ylim((mag_dat["period"].min(), mag_dat["period"].max()))
    ax[0].scatter(mag_dat["temperature"], mag_dat["period"], c=temp_loopbacks, s=1)
    
    ax[1].set_title("Corrected Temperature vs Period between\n" + str(mag_dat.index.min()) + " and " + str(mag_dat.index.max()))
    ax[1].set_ylim((mag_dat["period"].min(), mag_dat["period"].max()))
    ax[1].scatter(mag_dat["temp_corr"],mag_dat["period"], c=temp_loopbacks, s=1)
    mag_dat["temp_fit"] = fun(mag_dat["temp_corr"], *popt)
    mag_dat.sort_values("temp_corr", inplace=True)
    ax[1].plot(mag_dat["temp_corr"], mag_dat["temp_fit"])
    plt.show()
    mag_dat.drop("temp_fit", axis=1, inplace=True)
    
    
def get_mag_field(mag_dat, temp_fun=quad_fun, plots=True): 
    mag_dat["temp_corr"], temp_loopbacks = corr_temp(mag_dat)
    
    popt, pcov, mag_dat["period_temp_corr"] = corr_period_temp(mag_dat, temp_fun)
    print(popt)
    mag_dat["F"] = mag_field(mag_dat["period_temp_corr"])
    mag_dat["F"] -= mag_dat["F"].mean()
    
    if plots:
        plot_results(mag_dat, popt, temp_loopbacks, temp_fun)
    
    return mag_dat

for i in range(split_times.shape[0] - 1):
    try:
        all_mag_dat.loc[split_times[i]:split_times[i+1]] = get_mag_field(all_mag_dat.loc[split_times[i]:split_times[i+1]], plots=False)   
    except (ValueError, TypeError, RuntimeError) as e:
        print(e)
    
## Get IGRF values.
# Open the frongoch sensor details.
with open("frongoch_sensor_details.json", 'r') as frg_inf_file:
    frg_info = json.load(frg_inf_file)

# Downsample, because igrf results don't significantly over 1 day or less and the model is slow to process.
mag_dat_1d_times = all_mag_dat.resample("1d").first()
# Get results from model. This only selects the magnitude of the magnetic field from the data.
mag_dat_1d_times["igrf"] = mag_dat_1d_times.index.map(lambda dt: 
                                                      float(igrf12.igrf(dt,
                                                                      glat=frg_info["latitude"],
                                                                      glon=frg_info["longitude"],
                                                                      alt_km=frg_info["elevation"]/1000)["total"])).astype(np.float32)
# Upsample, and add to original data
all_mag_dat["igrf"] = mag_dat_1d_times["igrf"].resample('1T').ffill()

16:34:49 INFO cpDetector: =======================================
16:34:49 INFO cpDetector: Running change point detector
16:34:49 INFO cpDetector: =======================================
16:34:49 INFO cpDetector:    input observations: 1 of length [25946]
16:34:49 INFO cpDetector: Running cp detector on traj 0
16:34:49 INFO cpDetector: ---------------------------------
16:35:36 INFO cpDetector:     Found a new change point at: 8741!!
16:35:46 INFO cpDetector:     Found a new change point at: 7264!!
16:35:55 INFO cpDetector:     Found a new change point at: 2846!!
16:35:58 INFO cpDetector:     Found a new change point at: 325!!
16:36:01 INFO cpDetector:     Found a new change point at: 2512!!
16:36:03 INFO cpDetector:     Found a new change point at: 965!!
16:36:05 INFO cpDetector:     Found a new change point at: 1458!!
16:36:06 INFO cpDetector:     Found a new change point at: 1203!!
16:36:06 INFO cpDetector:     Found a new change point at: 1398!!
16:36:08 INFO cpDetector:     Found

/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

[ 1.15301161e-10 -6.26616131e-09  5.68023145e-06]


/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[ 2.61263313e-10 -1.35302134e-08  5.92024355e-06]
[ 3.43064448e-11 -4.13700804e-09  5.79381265e-06]
[ 1.59300435e-10 -9.63982651e-09  5.87928158e-06]
[ 7.28076462e-10 -3.48194196e-08  6.16373611e-06]
[-2.04370299e-11 -2.47037555e-09  5.79837466e-06]
[-3.51865477e-09  1.53862277e-07  4.04767591e-06]
[ 1.52726392e-09 -6.56218887e-08  6.43697875e-06]
[ 3.73042661e-10 -1.71566004e-08  5.81035624e-06]
[-3.38098543e-10  1.23572393e-08  5.55137921e-06]
[-7.42107127e-11 -3.24956176e-10  5.70515271e-06]
[ 1.66923322e-09 -7.66297080e-08  6.54167462e-06]
[ 6.95139675e-10 -3.46205489e-08  6.08911164e-06]
[ 3.78727253e-10 -1.93868265e-08  5.90699756e-06]
[ 7.54337390e-10 -3.79818267e-08  6.13558398e-06]
[ 1.11965255e-09 -5.16072214e-08  6.25772490e-06]
[-2.57227561e-09  1.09506226e-07  4.50449837e-06]
[ 9.34142911e-11 -6.96077992e-09  5.77777067e-06]
[-7.70888348e-11  1.15232612e-09  5.68110923e-06]
[-7.90483551e-11 -5.57717780e-10  5.71863823e-06]
[-5.27282555e-11 -2.58878581e-09  5.76163041e-06]


/home/james/anaconda3/envs/mag_srv_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


[ 4.45248656e-08 -2.00741437e-06  2.88519509e-05]
[ 1.69479943e-08 -1.19255515e-06  2.58161254e-05]
[ 1.99775854e-10 -1.28013764e-08  6.41658622e-06]
[-1.20101425e-09  6.14215871e-08  5.62547975e-06]
Improper input: N=3 must not exceed M=2
[-2.08031087e-05  1.20661804e-03 -1.74897328e-02]
[-2.32604834e-10  5.56416346e-09  6.22745069e-06]
[ 1.40608202e-10 -1.32761104e-08  6.76261177e-06]
[ 2.94071988e-09 -9.98488429e-08  7.42562882e-06]
[ 5.08371134e-10 -2.75688405e-08  6.90099097e-06]
[ 1.49838035e-10 -1.46166962e-08  6.79322430e-06]
[ 7.95757631e-11 -8.95805896e-09  5.91594767e-06]
[ 1.43868149e-10 -1.21179617e-08  5.96571309e-06]
[ 8.04749833e-11 -9.11393634e-09  5.92812109e-06]
[ 1.50961272e-10 -1.16266142e-08  5.94561910e-06]
[ 1.06810546e-10 -9.85111323e-09  5.91872222e-06]
[-8.01727326e-10  2.17813492e-08  5.64383555e-06]
[ 1.23730128e-10 -1.03891794e-08  5.91865033e-06]
[ 6.90018478e-11 -8.05617937e-09  5.90181559e-06]
[-6.95307128e-13 -4.38878834e-09  5.85723824e-06]
[-1.647035

Insert processed Frongoch magnetometer data to the database.

In [6]:
frg_insert.write_mag_dat(all_mag_dat[["F", "igrf"]].reset_index().to_dict('records'))

Now that the data is loaded, the [server](../server), for retrieving magnetometer data, needs setting up.